In [4]:
import pandas as pd
import os

In [12]:
# pd.read_table('car.data', delim_whitespace=True)
car_data = pd.read_csv('car.data')


In [20]:
car_data.describe()

,vhigh,vhigh.1,2,2.1,small,low,unacc
count,1727,1727,1727,1727,1727,1727,1727
unique,4,4,4,3,3,3,4
top,high,high,5more,4,med,high,unacc
freq,432,432,432,576,576,576,1209
